# Varol Multilabel Scraping

In [2]:
import pandas as pd
import tweepy
from IPython.display import clear_output
import time
from pandas.compat import StringIO

In [3]:
# create connection with Twitter API

CONSUMER_KEY = 'GeWIh6LTIiPd2mgMh4PhesKkX'
CONSUMER_SECRET = 'BjfHQp3T6XY3esXtYWo7mEh58zUd4v7USruqT1brFTXC0qOUYF'
ACCESS_TOKEN = '973862126216499200-tr3zhK2hOHnnhqegZMviFpoUj3T9nHa'
ACCESS_TOKEN_SECRET = 'LuHRMgpoGT3kjR95z8XPuafllrCaWRHyu86Pdru4zdATD'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [4]:
varol = pd.read_csv('../classification/data/bot_or_not/legitimate_users.txt', sep='\t')

In [11]:
varol = varol.rename(index=str, columns={"614": "id"})

In [13]:
user = api.get_user(varol.id[0])

varol_df = pd.DataFrame([[user.id,
                    user.name,
                    user.screen_name,
                    user.statuses_count,
                    user.followers_count,
                    user.friends_count,
                    user.favourites_count,
                    user.listed_count,
                    user.url,
                    user.lang,
                    user.time_zone,
                    user.location,
                    user.default_profile,
                    user.default_profile_image,
                    user.geo_enabled,
                    user.profile_image_url,
                    #user.profile_banner_url,
                    user.profile_use_background_image,
                    user.profile_background_image_url_https,
                    user.profile_text_color,
                    user.profile_image_url_https,
                    user.profile_sidebar_border_color,
                    user.profile_background_tile,
                    user.profile_sidebar_fill_color,
                    user.profile_background_image_url,
                    user.profile_background_color,
                    user.profile_link_color,
                    user.utc_offset,
                    user.is_translator,
                    user.follow_request_sent,
                    user.protected,
                    user.verified,
                    user.notifications,
                    user.description,
                    user.contributors_enabled,
                    user.following,
                    user.created_at,
                    0]],
                    columns=["id","name","screen_name","statuses_count","followers_count","friends_count","favourites_count","listed_count","url","lang","time_zone","location","default_profile","default_profile_image","geo_enabled","profile_image_url","profile_use_background_image","profile_background_image_url_https","profile_text_color","profile_image_url_https","profile_sidebar_border_color","profile_background_tile","profile_sidebar_fill_color","profile_background_image_url","profile_background_color","profile_link_color","utc_offset","is_translator","follow_request_sent","protected","verified","notifications","description","contributors_enabled","following","created_at", "target"]
                  )

In [ ]:
i = 0
error_ids_varol = []
error_count = 0

for index,row in varol[1:].iterrows():
    i = i + 1
    try:
        user = api.get_user(row.id)
        df2 = pd.DataFrame([[user.id,
                            user.name,
                            user.screen_name,
                            user.statuses_count,
                            user.followers_count,
                            user.friends_count,
                            user.favourites_count,
                            user.listed_count,
                            user.url,
                            user.lang,
                            user.time_zone,
                            user.location,
                            user.default_profile,
                            user.default_profile_image,
                            user.geo_enabled,
                            user.profile_image_url,
                            # user.profile_banner_url,
                            user.profile_use_background_image,
                            user.profile_background_image_url_https,
                            user.profile_text_color,
                            user.profile_image_url_https,
                            user.profile_sidebar_border_color,
                            user.profile_background_tile,
                            user.profile_sidebar_fill_color,
                            user.profile_background_image_url,
                            user.profile_background_color,
                            user.profile_link_color,
                            user.utc_offset,
                            user.is_translator,
                            user.follow_request_sent,
                            user.protected,
                            user.verified,
                            user.notifications,
                            user.description,
                            user.contributors_enabled,
                            user.following,
                            user.created_at,
                            0]],
                            columns=["id","name","screen_name","statuses_count","followers_count","friends_count","favourites_count","listed_count","url","lang","time_zone","location","default_profile","default_profile_image","geo_enabled","profile_image_url","profile_use_background_image","profile_background_image_url_https","profile_text_color","profile_image_url_https","profile_sidebar_border_color","profile_background_tile","profile_sidebar_fill_color","profile_background_image_url","profile_background_color","profile_link_color","utc_offset","is_translator","follow_request_sent","protected","verified","notifications","description","contributors_enabled","following","created_at","target"]
                          )
        varol_df = varol_df.append(df2, ignore_index=True, verify_integrity=False)
        
    except Exception as e:
        error_ids_varol.append(row)
        error_count += 1
        
    clear_output()
    print(str(i) + " / " + str(len(varol.id)) + '   ---   errors: ' + str(error_count))
    
    if i % 500 == 0:
        print ('sleeping...')
        time.sleep(900)
    

2720 / 19275   ---   errors: 294


In [30]:
tweets = []
cont, errors = 0, 0

for index, row in varol_df.iterrows():
    try:
        stuff = api.user_timeline(user_id = row.id, count = 100, include_rts = True, tweet_mode="extended")
        #tweet.is_quote_status indica se nel tweet c'è il riferimento ad un altro tweet

        for tweet in stuff:
            tweet._json['user_id'] = tweet._json['user']['id']
            if len(tweet._json['entities']['urls']) != 0:
                tweet._json['url'] = tweet._json['entities']['urls'][0]['expanded_url']
            else:
                 tweet._json['url'] = None
            del tweet._json['user'], tweet._json['entities']

            tweets.append(tweet._json)
    except:
        errors += 1
    cont += 1
    clear_output()
    print(str(cont) + ' / ' + str(len(varol_df)) + '   ---   errors: ' + str(errors))
    
    if cont % 500 == 0:
        print ('sleeping...')
        time.sleep(900)

706 / 706   ---   errors: 16


In [31]:
tweets_df = pd.DataFrame.from_dict(tweets)

In [32]:
# rimuove tutte le interruzioni (spero)

tweets_df['full_text'] = tweets_df['full_text'].apply(lambda x: x.replace('\r', ' '))
tweets_df['full_text'] = tweets_df['full_text'].apply(lambda x: x.replace('\n', ' '))
tweets_df['full_text'] = tweets_df['full_text'].apply(lambda x: x.replace('\t', ' '))

In [33]:
tweets_df.to_csv('../classification/data/varol/tweets.csv', index=False, sep='\t' , encoding='utf-8-sig')

In [34]:
varol_df.to_csv('../classification/data/varol/users.csv',index=False, encoding='utf-8-sig')

In [37]:
varol_df.loc[0]

id                                                                           3098421349
name                                                               Chem-DrySimcoeCounty
screen_name                                                              cdsimcoecounty
statuses_count                                                                     2081
followers_count                                                                     278
friends_count                                                                       917
favourites_count                                                                     88
listed_count                                                                          5
url                                                             https://t.co/l7eYicLoIA
lang                                                                                 en
time_zone                                                                          None
location                        